In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

def fetch_close_prices(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=False, progress=False)
    df = df[df.index <= pd.to_datetime(end_date)]
    return df['Close']

def compute_volatility(symbol, end_date, num_returns):
    buffer_days = int(num_returns * 1.5)
    start_date = pd.to_datetime(end_date) - pd.DateOffset(days=buffer_days)

    closes = fetch_close_prices(symbol, start_date, end_date)
    closes = closes.tail(num_returns + 1)  # N+1 prices = N returns

    returns = closes.pct_change().dropna()  # simple return

    # Convert to float using .item() which guarantees a scalar
    daily_sd = returns.std(ddof=0).item()
    annual_vol = (daily_sd * np.sqrt(252)).item()

    return round(daily_sd, 4), round(annual_vol, 4)

# Parameters
symbol = "NVDA"
end_date = "2025-04-01"

# Compute volatility for each period
sd_270, vol_270 = compute_volatility(symbol, end_date, 270)
sd_180, vol_180 = compute_volatility(symbol, end_date, 180)
sd_90,  vol_90  = compute_volatility(symbol, end_date, 90)

# Output formatted exactly like Excel screenshot
print("270")
print(f"Daily S.D. {sd_270:.2%}")
print(f"Annualized Volatility {vol_270:.2%}")

print("180")
print(f"Daily S.D. {sd_180:.2%}")
print(f"Annualized Volatility {vol_180:.2%}")

print("Number of days 90")
print(f"Daily SD {sd_90:.2%}")
print(f"Annualized Volatility {vol_90:.2%}")
